In [1]:
# STEP: IMPORT FUNCTIONS
import pandas as pd
from data_gathering import gather_data_features, gather_data_actuals
from data_exploration import calculate_correlation
from mappings import import_country_mapping
from data_visualization import global_scatter_plot

In [2]:
# STEP: LOAD DATA
# Load actuals data
data_cm_actual_2018, data_cm_actual_2019, data_cm_actual_2020, data_cm_actual_2021, data_cm_actual_allyears \
    = gather_data_actuals()

# Load features data
data_cm_features_2017, data_cm_features_2018, data_cm_features_2019, data_cm_features_2020, data_cm_features_allyears \
    = gather_data_features()


output_directory = r"C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\Baseline_Model"

crps_scores_all_year_country_specific = {}
crps_scores_all_year_global = {}
for year in [2015, 2016, 2017, 2018, 2019]:
    file_path = rf'C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\Baseline_Model\crps_scores_all_year_{year}_country_specific.parquet'
    crps_scores_all_year_country_specific[year] = pd.read_parquet(file_path)
    file_path = rf'C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\Baseline_Model\crps_scores_all_year_{year}_global.parquet'
    crps_scores_all_year_global[year] = pd.read_parquet(file_path)

crps_scores_all_year_country_specific_year_2015 = crps_scores_all_year_country_specific[2015]
crps_scores_all_year_country_specific_year_2016 = crps_scores_all_year_country_specific[2016]
crps_scores_all_year_country_specific_year_2017 = crps_scores_all_year_country_specific[2017]
crps_scores_all_year_country_specific_year_2018 = crps_scores_all_year_country_specific[2018]
crps_scores_all_year_country_specific_year_2019 = crps_scores_all_year_country_specific[2019]

crps_scores_all_year_global_year_2015 = crps_scores_all_year_global[2015]
crps_scores_all_year_global_year_2016 = crps_scores_all_year_global[2016]
crps_scores_all_year_global_year_2017 = crps_scores_all_year_global[2017]
crps_scores_all_year_global_year_2018 = crps_scores_all_year_global[2018]
crps_scores_all_year_global_year_2019 = crps_scores_all_year_global[2019]

country_mapping = import_country_mapping()

In [ ]:
# STEP: DATA EXPLORATION
input_variables = \
    data_cm_features_allyears.columns.drop(['index', 'country_id', 'month_id', 'ged_sb']).tolist()
target_variables = \
    ['ged_sb', 'ged_sb_tlag_1', 'ged_sb_tlag_2', 'ged_sb_tlag_3', 'ged_sb_tlag_4', 'ged_sb_tlag_5', 'ged_sb_tlag_6']
correlation_values = calculate_correlation(data_cm_features_allyears, target_variables=target_variables, input_variables=input_variables)



In [ ]:
data_cm_features_allyears

In [ ]:
data_cm_actual_allyears

In [ ]:
# Create a boolean mask of values with absolute value greater than or equal to 0.1
corr_threshold = 0.1
mask = correlation_values.abs() >= corr_threshold

# Apply the mask to the DataFrame to drop the values
correlation_values_filtered = correlation_values[mask]

correlation_values_filtered

**Understand spatial lag features**

In [ ]:

country_neighbours = [1, 2, 4, 9]
data_cm_features_allyears[['month_id', 'country_id', 'vdem_v2x_libdem', 'vdem_v2x_libdem_48', 'splag_vdem_v2x_libdem']][(data_cm_features_allyears['country_id'].isin(country_neighbours)) & (data_cm_features_allyears['month_id'] == 121)]

**Understand decay function features**

In [ ]:
country_neighbours = [1, 2, 4, 9]
data_cm_features_allyears[['month_id', 'country_id', 'ged_sb', 'decay_ged_sb_5', 'decay_ged_sb_100', 'decay_ged_sb_500']][(data_cm_features_allyears['country_id'] == 133)]

In [ ]:
country_neighbours = [1, 2, 4, 9]
data_cm_features_allyears[['month_id', 'country_id', 'ged_sb', 'decay_ged_sb_5', 'splag_1_decay_ged_sb_5' ]][(data_cm_features_allyears['country_id'].isin(country_neighbours)) & (data_cm_features_allyears['month_id'] == 121)]

Plot independent variables vs conflict fatalities in scatter plot

In [ ]:
output_directory = r'C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\Plots\scatterplots_global'
independent_variables = data_cm_features_allyears.columns.drop(['index', 'country_id', 'month_id', 'ged_sb']).tolist()
for independent_variable in independent_variables:
    global_scatter_plot(data_cm_features_allyears, independent_variable, 'ged_sb', output_directory, show=False, export=True)

Plot independent variables vs conflict fatalities in scatter plot for non zero conflict data

In [ ]:
output_directory_non_zero = r'C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\Plots\scatterplots_global_non_zero'
independent_variables = data_cm_features_allyears.columns.drop(['index', 'country_id', 'month_id', 'ged_sb']).tolist()
data_cm_features_allyears_non_zero = data_cm_features_allyears[data_cm_features_allyears['ged_sb'] > 0]
for independent_variable in independent_variables:
    global_scatter_plot(data_cm_features_allyears_non_zero, independent_variable, 'ged_sb', output_directory_non_zero, show=False, export=True)

In [ ]:
data_cm_features_allyears['ged_sb'].describe()


In [ ]:
data_cm_features_allyears_non_zero['ged_sb'].describe()

In [ ]:
data_cm_features_allyears[['month_id', 'country_id', 'ged_sb', 'ged_sb_tlag_1', 'ged_sb_tlag_2', 'ged_sb_tlag_3', 'ged_sb_tlag_4', 'ged_sb_tlag_5', 'ged_sb_tlag_6']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def plot_variable_distribution(df: pd.DataFrame, variable: str):
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(data=df, x=variable, ax=ax)
    ax.set_title(f'Distribution of {variable}')
    plt.show()

In [ ]:
plot_variable_distribution(data_cm_features_allyears, 'ged_sb')

In [ ]:
plot_variable_distribution(data_cm_features_allyears_non_zero, 'ged_sb')

In [ ]:
global_scatter_plot(data_cm_features_allyears_non_zero, 'wdi_sm_pop_refg_or', 'ged_sb', output_directory, show=True, export=False)

In [ ]:
data_cm_features_allyears_non_zero_big =  data_cm_features_allyears[data_cm_features_allyears['ged_sb'] > 5]

In [ ]:
plot_variable_distribution(data_cm_features_allyears_non_zero_big, 'ged_sb')

In [ ]:
import pymc as pm

In [ ]:
run_id = 'eae5db7dfa6e4408a8fb0f916968d033'
evaluation_year = 2018
results_all_countries_path = fr"C:\Users\Uwe Drauz\PycharmProjects\bachelor_thesis\mlruns\3\{run_id}\artifacts\BaselineModel_results_all_countries{evaluation_year}.parquet"
results_all_countries = pd.read_parquet(results_all_countries_path)
results_actuals_all_countries_path = fr"C:\Users\Uwe Drauz\PycharmProjects\bachelor_thesis\mlruns\3\{run_id}\artifacts\BaselineModel_results_actuals_{evaluation_year}.parquet"
results_actuals = pd.read_parquet(results_actuals_all_countries_path)

### Prior Predicitve Checks

In [ ]:
import scipy.stats as stats
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

# Simulate gamma distribution of alpha parameter
nsim = 100
nobs = 100
a_alpha = 0.1
b_alpha = 1

# Generate alpha using Gamma distribution
alpha = stats.gamma.rvs(a=a_alpha, scale=1/b_alpha, size=nsim)

# Initialize DataFrame to store Negative Binomial data
y = pd.DataFrame(index=range(nsim), columns=range(nobs))

# Simulate target data following NB distribution
mu = 1
for i in range(nsim):
    # Generate Negative Binomial data using the alpha parameter
    y.loc[i, :] = stats.nbinom.rvs(n=alpha[i], p=mu/(mu+alpha[i]), size=nobs)




In [ ]:
# Plot the histogram using Seaborn
sns.histplot(alpha, bins=30, label='Histogram', color='blue')

# Create a second y-axis to plot the density
ax2 = plt.gca().twinx()

# Plot the density on the second y-axis
sns.kdeplot(alpha, ax=ax2, label='Density', color='red')

# Labeling and titles
plt.title('Histogram and Density of Alpha')
plt.xlabel('Alpha')
plt.gca().set_ylabel('Frequency (Count)')
ax2.set_ylabel('Density')

# Add legends
plt.gca().legend(loc='upper left')
ax2.legend(loc='upper right')

plt.show()


In [ ]:
# Plot the histogram using Seaborn for the first row of y
for obs in range(20):
    print(f"alpha value: {alpha[obs]}")
    sns.histplot(y.loc[obs, :], bins=30, label='Histogram', color='blue')
    
    # Create a second y-axis to plot the density
    ax2 = plt.gca().twinx()
    
    # Plot the density on the second y-axis for the first row of y
    sns.kdeplot(y.loc[obs, :], ax=ax2, label='Density', color='red')
    
    # Labeling and titles
    plt.title('Histogram and Density of y (Row 0)')
    plt.xlabel('y')
    plt.gca().set_ylabel('Frequency (Count)')
    ax2.set_ylabel('Density')
    
    # Add legends
    plt.gca().legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.show()


In [ ]:
# Load 'Actuals' data
data_cm_actual_2018, data_cm_actual_2019, data_cm_actual_2020, data_cm_actual_2021, data_cm_actual_allyears \
    = gather_data_actuals()
# Load features data
data_cm_features_2017, data_cm_features_2018, data_cm_features_2019, data_cm_features_2020, data_cm_features_allyears \
    = gather_data_features()

In [ ]:
dict_actuals = {
        "2018": data_cm_actual_2018,
        "2019": data_cm_actual_2019,
        "2020": data_cm_actual_2020,
        "2021": data_cm_actual_2021
        }

In [ ]:
# Determine all countries
actual_countries = data_cm_actual_allyears['country_id'].unique()
# Determine countries with at least one conflict fatality
feature_countries_non_zero = data_cm_features_allyears[data_cm_features_allyears['ged_sb'] > 0][
    'country_id'].unique()
# Determine countries which have at least one conflict fatality and are in the actuals data
feature_and_actuals_countries_non_zero = list(set(feature_countries_non_zero) & set(actual_countries))

# Determine the counties which are in actual_countries but not in feature_and_actuals_countries_non_zero
countries_in_actuals_without_observations = list(
    set(actual_countries) - set(feature_and_actuals_countries_non_zero))

In [ ]:
# Calculate the average value of 'ged_sb' for each data frames in dict_actuals filtered on countries which are in feature_and_actuals_countries_non_zero
for year in dict_actuals.keys():
    df = dict_actuals[year]
    df = df[df['country_id'].isin(feature_and_actuals_countries_non_zero)]
    print(f"Average value of 'ged_sb' in {year}: {df['ged_sb'].mean()}")

In [ ]:
X = pd.read_parquet("competition_results/test_window_2018_baseline_validate_on_forecast_horizon.parquet")

In [ ]:
from mappings import map_month_id_to_datetime
map_month_id_to_datetime(468)

In [ ]:
import pandas as pd
results_all_countries_2018 = pd.read_parquet("mlruns/3/bd79953d84ac4d3d8d59415a286c3fc3/artifacts/BaselineModel_results_all_countries2018.parquet")

### Categorizing countries into developed and developing countries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# List of countries in the views competition data
views_country_list = [
    "Guyana", "Suriname", "Trinidad and Tobago", "Venezuela", "Samoa", "Tonga",
    "Argentina", "Bolivia", "Brazil", "Chile", "Ecuador", "Paraguay", "Peru", "Uruguay",
    "Guatemala", "Mexico", "Barbados", "Dominica", "Grenada", "St. Lucia", "St. Vincent and the Grenadines",
    "Dominican Republic", "Haiti", "Jamaica", "Bahamas", "Belize", "Colombia", "Costa Rica", "Cuba",
    "El Salvador", "Honduras", "Nicaragua", "Panama", "Antigua and Barbuda", "St. Kitts and Nevis",
    "Iceland", "Ireland", "United Kingdom", "Cape Verde", "Cote d'Ivoire", "Ghana", "Liberia",
    "Portugal", "Spain", "Burkina Faso", "Guinea", "Guinea-Bissau", "Mali",
    "Senegal", "Sierra Leone", "The Gambia", "Djibouti", "Eritrea", "Ethiopia", "Mongolia",
    "Iraq", "Jordan", "Kazakhstan", "Norway", "Russia", "Sweden",
    "Algeria", "Cameroon", "Central African Republic", "Tunisia", "Benin",
    "Equatorial Guinea", "Kiribati", "Niger", "Nigeria", "Sao Tome and Principe", "Togo",
    "Albania", "Bosnia and Herzegovina", "Croatia", "Italy", "Macedonia", "Malta",
    "Bulgaria", "Cyprus", "Georgia", "Greece", "Lebanon",
    "Turkey", "Austria", "Czech Republic", "Denmark", "Hungary", "Poland",
    "Slovakia", "Slovenia", "Belgium", "France", "Luxembourg", "Netherlands",
    "Switzerland", "Belarus", "Estonia", "Finland", "Latvia", "Lithuania", "Moldova",
    "Romania", "Ukraine", "Maldives", "Oman", "Somalia", "Sri Lanka", "Turkmenistan",
    "Uzbekistan", "Yemen", "Armenia", "Azerbaijan", "Bahrain", "Iran", "Kuwait",
    "Qatar", "Saudi Arabia", "United Arab Emirates", "Afghanistan", "Kyrgyzstan",
    "Nepal", "Pakistan", "Tajikistan", "Bangladesh", "Bhutan", "Brunei",
    "Japan", "North Korea", "Palau", "Philippines", "South Korea",
    "Cambodia", "Laos", "Myanmar", "Thailand", "Vietnam", "Marshall Is.",
    "Micronesia", "Botswana", "Burundi", "Rwanda", "Zambia", "Zimbabwe",
    "Comoros", "Lesotho", "Malawi", "Mozambique", "South Africa", "Swaziland",
    "Angola", "Congo", "Congo, DRC", "Fiji", "Gabon", "Namibia",
    "New Zealand", "Madagascar", "Mauritius", "Seychelles", "Timor Leste",
    "Australia", "Nauru", "Papua New Guinea", "Solomon Is.", "Tuvalu",
    "Vanuatu", "Canada", "Germany", "Taiwan", "China", "Malaysia",
    "Singapore", "Indonesia", "Libya", "Chad", "Israel", "Syria",
    "Egypt", "India", "Montenegro", "Kosovo", "Serbia", "United States",
    "Uganda", "Kenya", "Tanzania", "Morocco", "Mauritania", "Sudan", "South Sudan"
]


**List of categorized countries based on the World Economic Situation and Prospects (WESP) 2022 report**
To be found here: https://www.un.org/development/desa/dpad/wp-content/uploads/sites/45/WESP2022_ANNEX.pdf

In [ ]:
# List of developed economies from Table A
developed_economies = [
    "Canada", "United States", "Australia", "Japan", "New Zealand",
    "Austria", "Belgium", "Denmark", "Finland", "France", "Germany",
    "Greece", "Ireland", "Italy", "Luxembourg", "Netherlands", "Portugal",
    "Spain", "Sweden", "Bulgaria", "Croatia", "Cyprus", "Czech Republic",
    "Estonia", "Hungary", "Latvia", "Lithuania", "Malta", "Poland",
    "Romania", "Slovakia", "Slovenia", "Iceland", "Norway", "Switzerland",
    "United Kingdom"
]

In [ ]:
# List of economies in transition from Table B
economies_in_transition = [
    "Albania", "Bosnia and Herzegovina", "Montenegro", "Macedonia", "Serbia",
    "Armenia", "Azerbaijan", "Belarus", "Georgia", "Kazakhstan", "Kyrgyzstan",
    "Moldova", "Russia", "Tajikistan", "Turkmenistan",
    "Ukraine", "Uzbekistan"
]

In [ ]:
# List of developing economies from Table C
developing_economies = [
    # North Africa
    "Algeria", "Egypt", "Libya", "Mauritania", "Morocco", "Sudan", "Tunisia",
    # Central Africa
    "Cameroon", "Central African Republic", "Chad", "Congo, DRC", "Equatorial Guinea", "Gabon", "Sao Tome and Prinicipe",
    # East Africa
    "Burundi", "Congo", "Comoros", "Djibouti", "Eritrea", "Ethiopia", "Kenya", "Madagascar", "Rwanda", "Somalia", "South Sudan", "Tanzania", "Uganda",
    # Southern Africa
    "Angola", "Botswana", "Swaziland", "Lesotho", "Malawi", "Mauritius", "Mozambique", "Namibia", "South Africa", "Zambia", "Zimbabwe",
    # West Africa
    "Benin", "Burkina Faso", "Cape Verde", "Cote d'Ivoire", "The Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Liberia", "Mali", "Niger", "Nigeria", "Senegal", "Sierra Leone", "Togo",
    # Asia
    "Brunei", "Cambodia", "China", "North Korea", "Fiji", "Hong Kong SAR", "Indonesia", "Kiribati", "Laos", "Malaysia", "Mongolia", "Myanmar", "Papua New Guinea", "Philippines", "South Korea", "Samoa", "Singapore", "Solomon Is.", "Taiwan", "Thailand", "Timor-Leste", "Vanuatu", "Vietnam",
    "Afghanistan", "Bangladesh", "Bhutan", "India", "Iran", "Maldives", "Nepal", "Pakistan", "Sri Lanka",
    "Bahrain", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", "Oman", "Qatar", "Saudi Arabia", "State of Palestine", "Syria", "Turkey", "United Arab Emirates", "Yemen",
    # Latin America and the Caribbean
    "Bahamas", "Barbados", "Belize", "Guyana", "Jamaica", "Suriname", "Trinidad and Tobago", "Costa Rica", "Cuba", "Dominican Republic", "El Salvador", "Guatemala", "Haiti", "Honduras", "Mexico", "Nicaragua", "Panama",
    "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Paraguay", "Peru", "Uruguay", "Venezuela"
]


In [ ]:
# List of the least developed economies from Table F based on the UN list of LDCs
least_developed_countries = [
    "Angola", "Benin", "Burkina Faso", "Burundi", "Central African Republic",
    "Chad", "Comoros", "Congo, DRC", "Djibouti", "Eritrea",
    "Ethiopia", "The Gambia", "Guinea", "Guinea-Bissau", "Lesotho", "Liberia",
    "Madagascar", "Malawi", "Mali", "Mauritania", "Mozambique", "Niger",
    "Rwanda", "Sao Tome and Principe", "Senegal", "Sierra Leone", "Somalia",
    "South Sudan", "Sudan", "Togo", "Uganda", "Tanzania",
    "Zambia", "Cambodia", "Kiribati", "Laos",
    "Myanmar", "Solomon Is.", "Timor Leste", "Tuvalu", "Afghanistan",
    "Bangladesh", "Bhutan", "Nepal", "Yemen", "Haiti"
]

In [ ]:
# List of small developing island states from Table H based on the UN list of SIDS
small_islands_developing_states = [
    "Antigua and Barbuda", "Bahamas", "Bahrain", "Barbados", "Belize", "Cape Verde", "Comoros", "Cuba", "Dominica", "Dominican Republic", "Micronesia", "Fiji", "Grenada", "Guinea-Bissau", "Guyana", "Haiti", "Jamaica", "Kiribati",  "Maldives", "Marshall Is.", "Mauritius", "Nauru", "Palau", "Papua New Guinea", "St. Kitts and Nevis", "St. Lucia", "St. Vincent and the Grenadines", "Samoa", "Sao Tome and Principe", "Seychelles", "Singapore", "Solomon Is.", "Suriname", "Timor-Leste", "Tonga", "Trinidad and Tobago", "Tuvalu", "Vanuatu", "American Samoa", "Anguilla", "Aruba", "Bermuda", "British Virgin Islands", "Cayman Islands", "Commonwealth of Northern Marianas", "Cook Islands", "Curaçao", "French Polynesia", "Guadeloupe", "Guam", "Martinique", "Montserrat", "New Caledonia", "Niue", "Puerto Rico", "Sint Maarten", "Turks and Caicos Islands", "U.S. Virgin Islands"
]


In [ ]:
# not fully recognized countries
not_recognized_countries = ['Kosovo']

Construct different sets of countries 

In [ ]:
# Combined economies types from WESP 2022 report
all_economies_wesp = developed_economies + economies_in_transition + developing_economies

In [ ]:
# Combined economies types from WESP 2022 report extended by LDCs and SIDS
all_economies_wesp_plus_UN = list(set(all_economies_wesp + least_developed_countries + small_islands_developing_states))

In [ ]:
# Developing economies from WESP 2022 report extended by LDCs, SIDS ans outlier countries
developing_economies_extended = developing_economies + least_developed_countries + small_islands_developing_states + not_recognized_countries
developing_economies_extended = list(set(developing_economies_extended))

In [ ]:
# List of developing economies without LDCs n
developing_economies_extended_without_LDC = list(set(developing_economies_extended) - set(least_developed_countries))

In [ ]:
df_all_countries = pd.DataFrame(all_economies_wesp_plus_UN, columns=['country'])
# Print countires in views_country_list which are not in all_economies_wesp_plus_UN
divergences = []
for country in views_country_list:
    if country not in all_economies_wesp_plus_UN:   
        divergences.append(country)
divergences_df = pd.DataFrame(divergences, columns=['country']); divergences_df

In [ ]:
def are_lists_distinct(*lists):
    all_elements = []
    for lst in lists:
        all_elements.extend(lst)
    return len(all_elements) == len(set(all_elements))

Explore Data of different country sets

In [ ]:
from mappings import import_country_mapping
from data_gathering import gather_data_features, gather_data_actuals
from data_preparation import preprocess_data

country_mapping = import_country_mapping()

In [ ]:
# Load features data
data_cm_features_2017, data_cm_features_2018, data_cm_features_2019, data_cm_features_2020, data_cm_features_allyears \
    = gather_data_features()

In [ ]:
# Retrieve country_ids for different country sets
developed_countries_ids = country_mapping[country_mapping['name'].isin(developed_economies)]['country_id'].tolist()
countries_in_transition_ids = country_mapping[country_mapping['name'].isin(economies_in_transition)]['country_id'].tolist()
developing_countries_extended_without_LDC_ids = country_mapping[country_mapping['name'].isin(developing_economies_extended_without_LDC)]['country_id'].tolist()
least_developed_countries_ids = country_mapping[country_mapping['name'].isin(least_developed_countries)]['country_id'].tolist()


In [ ]:
# Plot the distribution of the ged_sb variable for different country sets based on the features data
import seaborn as sns
import matplotlib.pyplot as plt
def plot_variable_distribution(df: pd.DataFrame, variable: str):
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.histplot(data=df, x=variable, ax=ax, kde=True)
    ax.set_title(f'Distribution of {variable}')
    plt.show()

In [ ]:
data_cm_features_allyears.describe()

In [ ]:
# Plot the distribution of the ged_sb variable over all countries
plot_variable_distribution(data_cm_features_allyears, 'ged_sb')
# Plot the distribution of the ged_sb variable over all countries
plot_variable_distribution(np.log(data_cm_features_allyears[data_cm_features_allyears['ged_sb'] > 0]), 'ged_sb')
# Plot the distribution of the ged_sb variable over all countries
plot_variable_distribution(data_cm_features_allyears[data_cm_features_allyears['ged_sb'] < 1000], 'ged_sb')
# Plot the distribution of the ged_sb variable over all countries
plot_variable_distribution(data_cm_features_allyears[data_cm_features_allyears['ged_sb'] >= 1000], 'ged_sb')

In [ ]:
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(developed_countries_ids))], 'ged_sb')
plot_variable_distribution(np.log(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(developed_countries_ids)) & (data_cm_features_allyears['ged_sb'] > 0)]), 'ged_sb')
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(developed_countries_ids)) & (data_cm_features_allyears['ged_sb'] < 1000)], 'ged_sb')
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(developed_countries_ids)) & (data_cm_features_allyears['ged_sb'] >= 1000)], 'ged_sb')

In [ ]:
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(countries_in_transition_ids))], 'ged_sb')
plot_variable_distribution(np.log(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(countries_in_transition_ids)) & (data_cm_features_allyears['ged_sb'] > 0)]), 'ged_sb')
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(countries_in_transition_ids)) & (data_cm_features_allyears['ged_sb'] < 1000)], 'ged_sb')
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(countries_in_transition_ids)) & (data_cm_features_allyears['ged_sb'] >= 1000)], 'ged_sb')

In [ ]:
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(developing_countries_extended_without_LDC_ids))], 'ged_sb')
plot_variable_distribution(np.log(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(developing_countries_extended_without_LDC_ids)) & (data_cm_features_allyears['ged_sb'] > 0)]), 'ged_sb')
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(developing_countries_extended_without_LDC_ids)) & (data_cm_features_allyears['ged_sb'] < 1000)], 'ged_sb')
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(developing_countries_extended_without_LDC_ids)) & (data_cm_features_allyears['ged_sb'] >= 1000)], 'ged_sb')

In [ ]:
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(least_developed_countries_ids))], 'ged_sb')
plot_variable_distribution(np.log(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(least_developed_countries_ids)) & (data_cm_features_allyears['ged_sb'] > 0)]), 'ged_sb')
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(least_developed_countries_ids)) & (data_cm_features_allyears['ged_sb'] < 1000)], 'ged_sb')
plot_variable_distribution(data_cm_features_allyears[(data_cm_features_allyears['country_id'].isin(least_developed_countries_ids)) & (data_cm_features_allyears['ged_sb'] >= 1000)], 'ged_sb')

In [ ]:
# Compute max and average values for ged_sb over all countries
print(f"Max value of ged_sb over all countries: {data_cm_features_allyears['ged_sb'].max()}")
print(f"Average value of ged_sb over all countries: {data_cm_features_allyears['ged_sb'].mean()}")

In [ ]:
# Compute max and average values for ged_sb on different country sets
print(f"Max value of ged_sb for developed countries: {data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(developed_countries_ids)]['ged_sb'].max()}")
print(f"Max value of ged_sb for countries in transition: {data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(countries_in_transition_ids)]['ged_sb'].max()}")
print(f"Max value of ged_sb for developing countries: {data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(developing_countries_extended_without_LDC_ids)]['ged_sb'].max()}")
print(f"Max value of ged_sb for least developed countries: {data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(least_developed_countries_ids)]['ged_sb'].max()}")

In [ ]:
print(f"Average value of ged_sb for developed countries: {data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(developed_countries_ids)]['ged_sb'].mean()}")
print(f"Average value of ged_sb for countries in transition: {data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(countries_in_transition_ids)]['ged_sb'].mean()}")
print(f"Average value of ged_sb for developing countries: {data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(developing_countries_extended_without_LDC_ids)]['ged_sb'].mean()}")
print(f"Average value of ged_sb for least developed countries: {data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(least_developed_countries_ids)]['ged_sb'].mean()}")

In [ ]:
# Select data based on country sets
df_developed_countries = data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(developed_countries_ids)]
df_countries_in_transition = data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(countries_in_transition_ids)]
df_developing_countries_extended_without_LDC = data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(developing_countries_extended_without_LDC_ids)]
df_least_developed_countries = data_cm_features_allyears[data_cm_features_allyears['country_id'].isin(least_developed_countries_ids)]


In [ ]:
# Store the set-specific data in a dictionary under the respective key
dict_country_sets = {
    "developed_countries": df_developed_countries,
    "countries_in_transition": df_countries_in_transition,
    "developing_countries_extended_without_LDC": df_developing_countries_extended_without_LDC,
    "least_developed_countries": df_least_developed_countries
}

In [ ]:
# Use the funtion "temporary_standardization" to standardize the data for each country set, and "temporary_feature_selection" to select the features for each country set
for key in dict_country_sets.keys():
    df = dict_country_sets[key]
    df = temporary_feature_selection(df)
    df = temporary_standardization(df)
    dict_country_sets[key] = df

In [ ]:
# Export the set-specific data to parquet files under the name "data_cm_features_allyears_{key}.parquet"
for key in dict_country_sets.keys():
    df = dict_country_sets[key]
    df.to_parquet(f"C:/Users/Uwe Drauz/Documents/bachelor_thesis_local/personal_competition_data/temp/data_cm_features_allyears_{key}.parquet")

In [ ]:
# Preprocess data
lagged_covariates = ["ged_sb", "ged_sb_tsum_24", "decay_ged_sb_5", "decay_ged_sb_100", "decay_ged_sb_500"]
important_vdem_features = ["vdem_v2x_veracc",
                          "vdem_v2x_horacc",
                          "vdem_v2xnp_client",
                          "vdem_v2x_divparctrl",
                          "vdem_v2xpe_exlpol",
                          "vdem_v2xpe_exlsocgr"]
important_wdi_features = ["wdi_ms_mil_xpnd_zs", 
                          "wdi_sm_pop_refg_or",
                          "wdi_sm_pop_netm",
                          "wdi_sp_pop_grow",
                          "wdi_dt_oda_odat_pc_zs"]
covariates =  important_vdem_features + important_wdi_features

In [ ]:
from importlib import reload



In [ ]:
data = preprocess_data(df=data_cm_features_allyears, covariates=covariates, lagged_covariates=lagged_covariates, standardize=True)

In [ ]:
data.to_parquet("C:/Users/Uwe Drauz/Documents/bachelor_thesis_local/personal_competition_data/temp/special_feature_selection_std.parquet")

In [ ]:
from data_visualization import global_scatter_plot

In [ ]:
# Plot the vdem and wdi indices against ged_sb
for vdem in important_vdem_features:
    global_scatter_plot(data, vdem, "ged_sb", output_dir="C:/Users/Uwe Drauz/Documents/bachelor_thesis_local/personal_competition_data/temp")

In [ ]:
for wdi in important_wdi_features:
    global_scatter_plot(data, wdi, "ged_sb", output_dir="C:/Users/Uwe Drauz/Documents/bachelor_thesis_local/personal_competition_data/temp")

In [ ]:
global_scatter_plot(data, "decay_ged_sb_100_tlag_3", "ged_sb", output_dir="C:/Users/Uwe Drauz/Documents/bachelor_thesis_local/personal_competition_data/temp")

In [ ]:
result_data_2021[result_data_2021['test_score'] > 2000].sort_values(by=['country_id', 'month_id'])

In [ ]:
result_data_2021[result_data_2021['test_score'] > 2000].sum()


In [ ]:
from mappings import map_month_id_to_datetime
from mappings import map_country_id_to_country_name

In [ ]:
import mappings
reload(mappings)
from mappings import map_country_id_to_country_name

In [ ]:
data_cm_actual_2018, data_cm_actual_2019, data_cm_actual_2020, data_cm_actual_2021, data_cm_actual_allyears \
    = gather_data_actuals()

In [ ]:
map_month_id_to_datetime(498)

In [ ]:
map_country_id_to_country_name(133)

In [ ]:
data_cm_actual_2021[(data_cm_actual_2021['country_id'] == 133)]

Rolling window lengths of month country_combination

Test Scores of month country_combination

In [47]:
result_data_2018_FH_test_score

,January 2018,February 2018,March 2018,April 2018,May 2018,June 2018,July 2018,August 2018,September 2018,October 2018,November 2018,December 2018
Trinidad and Tobago,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Venezuela,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Ecuador,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Peru,0.026218,0.030012,0.034696,0.023043,0.026218,0.030012,0.034696,0.040518,0.047865,0.057340,0.069789,0.034696
Guatemala,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
Tanzania,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Morocco,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Mauritania,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Sudan,3.688134,2.028846,18.129666,6.819406,25.122941,15.345552,19.019859,24.943496,19.869960,24.387940,19.137435,21.785516


### CRPS scores of Stan model

In [5]:
import pandas as pd
import CRPS.CRPS as crps

In [15]:
posterior_samples_zinb = pd.read_csv(r"C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\temp\posterior_samples_zinb.csv")
posterior_samples_nb = pd.read_csv(r"C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\temp\posterior_samples_nb.csv")
df_eval = pd.read_csv(r"C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\temp\df_eval.csv")

In [16]:
posterior_samples_zinb = posterior_samples_zinb.drop(columns=['Unnamed: 0'])
posterior_samples_nb = posterior_samples_nb.drop(columns=['Unnamed: 0'])

In [17]:
# Calculate CRPS score for each column in posterior_samples corresponding to the observation (column "ged_sb") in df_eval
crps_scores_zinb = pd.DataFrame(columns=['crps_score', 'country_id'])
for i in range(0, len(posterior_samples_zinb.columns)):
    posterior_samples_zinb_country = posterior_samples_zinb.iloc[:, i]
    observation = df_eval['ged_sb'][i]
    country_id = df_eval['country_id'][i]
    crps_score_zinb, _, _ = crps(posterior_samples_zinb_country, observation).compute()
    crps_scores_zinb = crps_scores_zinb.append({'crps_score': crps_score_zinb, 'country_id': country_id}, ignore_index=True)    

In [18]:
# Calculate CRPS score for each column in posterior_samples corresponding to the observation (column "ged_sb") in df_eval
crps_scores_nb = pd.DataFrame(columns=['crps_score', 'country_id'])
for i in range(0, len(posterior_samples_nb.columns)):
    posterior_samples_nb_country = posterior_samples_nb.iloc[:, i]
    observation = df_eval['ged_sb'][i]
    country_id = df_eval['country_id'][i]
    crps_score_nb, _, _ = crps(posterior_samples_nb_country, observation).compute()
    crps_scores_nb = crps_scores_nb.append({'crps_score': crps_score_nb, 'country_id': country_id}, ignore_index=True)    

In [19]:
# Calculate the mean CRPS score over all countries
crps_scores_zinb['crps_score'].mean()


64.07997440535021

In [20]:
crps_scores_nb['crps_score'].mean()

48.862956655092596

### Inspect neigbors data

In [1]:
import pandas as pd

In [5]:
neigbors_data = pd.read_csv(r"C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\shared_competition_data\country_neighbors\fallback3_prod_country_country_month.csv")

In [6]:
# Rename columns
neigbors_data = neigbors_data.rename(columns={'59': 'country_id_a', 'Sudan': 'country_a', '70': 'country_id_b', 'Central African Republic': 'country_b', '[0,379)': 'month_id_period'})
# Drop remaining columns
neigbors_data = neigbors_data.drop(columns=['1', '625', '482'])

In [9]:
# Drop all entries where month_id_period does not end with "612)"
neigbors_data_current = neigbors_data[neigbors_data['month_id_period'].str.endswith("612)")]

In [10]:
# Count unique country_id_a
neigbors_data_current['country_id_a'].nunique()

156

In [8]:
# count unique country_a
neigbors_data['country_a'].nunique()

164

Analysing data of geodata

In [12]:
neigbors_data_geodata = pd.read_csv(r"C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\shared_competition_data\country_neighbors\GEODATASOURCE-COUNTRY-BORDERS.csv")

In [14]:
# Get number of unique country names
neigbors_data_geodata['country_code'].nunique()

248

In [16]:
# Get number of unique country names for all observations where country_code is not null
neigbors_data_geodata[neigbors_data_geodata['country_border_code'].notnull()]['country_code'].nunique()

162

### Analyse share of structured zeros in fatalities

In [1]:
# Import packages
import pandas as pd
import numpy as np
from data_gathering import gather_data_features, gather_data_actuals

In [2]:
# Gather data
data_cm_actual_2018, data_cm_actual_2019, data_cm_actual_2020, data_cm_actual_2021, data_cm_actual_allyears \
    = gather_data_actuals()
data_cm_features_2018, data_cm_features_2019, data_cm_features_2020, data_cm_features_2021, data_cm_features_allyears \
    = gather_data_features()

In [6]:
# Calculate share of structured zeros in fatalities and average fatalities per month per country (fatalities = ged_sb)
data_cm_features_allyears['share_structured_zeros'] = data_cm_features_allyears[data_cm_features_allyears['ged_sb'] == 0].groupby('country_id')['ged_sb'].count() / data_cm_features_allyears.groupby('country_id')['ged_sb'].count()
data_cm_features_allyears['avg_fatalities_per_month'] = data_cm_features_allyears.groupby('country_id')['ged_sb'].mean()
desired_data = data_cm_features_allyears[['country_id', 'share_structured_zeros', 'avg_fatalities_per_month']].dropna().sort_values(by='share_structured_zeros', ascending=True).reset_index(drop=True)

In [15]:
# Calculate quantiles of share of structured zeros in fatalities and average fatalities per month per country (fatalities = ged_sb) excluding the countries with structured zeros share of 1
structured_zeros_quantiles = desired_data['share_structured_zeros'].quantile([0.25, 0.5, 0.75])
avg_fatalities_quantiles = desired_data['avg_fatalities_per_month'].quantile([0.25, 0.5, 0.75])

In [16]:
print(structured_zeros_quantiles)
print(avg_fatalities_quantiles)

0.25    0.830502
0.50    1.000000
0.75    1.000000
Name: share_structured_zeros, dtype: float64
0.25    0.000000
0.50    0.000000
0.75    5.289189
Name: avg_fatalities_per_month, dtype: float64


In [ ]:
# Select only the data fpr which the average fatalities per month is greater than the 0.75 quantile of average fatalities per month
data_cm_features_allyears_selected = data_cm_features_allyears[data_cm_features_allyears['avg_fatalities_per_month'] > avg_fatalities_quantiles[0.75]]

In [2]:
from mappings import map_month_id_to_datetime

In [3]:
map_month_id_to_datetime(468)

datetime.datetime(2018, 12, 1, 0, 0)